## **Off-Policy Evaluation (OPE) Methods in Reinforcement Learning**

References:
- [A Review of Off-Policy Evaluation in Reinforcement Learning - Masatoshi Uehara](https://arxiv.org/pdf/2212.06355)
- [discrete-off-policy-evaluation github implementation](https://github.com/theophilegervet/discrete-off-policy-evaluation)
- [Ch-9: Importance Sampling - Art Owen](https://artowen.su.domains/mc/Ch-var-is.pdf). The entire book is [Monte Carlo theory, methods and examples - Art Owen](https://artowen.su.domains/mc/)

In this notebook, we will implement 7 OPE methods:
- **IS**: Importance Sampling
- **WIS**: Weighted Importance Sampling
- **PDIS**: Per-Decision Importance Sampling
- **WPDIS**: Weighted Per-Decision Importance Sampling
- **FQE**: Fitted Q-Evaluation
- **DR**: Doubly Robust
- **WDR**: Weighted Doubly Robust

### **Core Problem and Notation**

**Goal:** We want to estimate the value (expected return) of a new evaluation policy ($\pi_e$) using data collected by an old behavior policy ($\pi_b$).

This is crucial for real-world problems (like a recommender system, robotics, or healthcare) where testing a new, potentially bad policy ($\pi_e$) live is too costly or dangerous.

Key Notation:
- **State** ($s$): The current situation or observation in the environment.
- **Action** ($a$): The decision or move made by the agent.
- **Reward** ($r$): The immediate feedback received after taking action $a$ in state $s$.
- **Policy** ($\pi$): A function $\pi(a|s)$ that gives the probability of taking action $a$ in state $s$.
- **Trajectory** ($\tau$): A sequence of states, actions, and rewards from an episode: $\displaystyle \tau = (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$.
- **Discount Factor** ($\gamma$): A number in $[0, 1)$ that reduces the importance of future rewards.
- **Return** ($G(\tau)$): The (discounted) sum of rewards for a trajectory: $\displaystyle G(\tau) = \sum_{t=0}^T \gamma^t r_t$. We often use $G_i$ for the return of trajectory $i$.
- **Value** ($V(\pi)$): The expected return of a policy: $\displaystyle V(\pi) = \mathbb{E}_{\tau \sim \pi}[G(\tau)]$. This is what we want to estimate for $\pi_e$.
    - Note: $\displaystyle \mathbb{E}_{a \sim b}[X]$ means the expected value of the random variable $X$, where the variable $a$ is sampled from the distribution $b$.
- **Behavior Policy** ($\pi_b$): The policy that generated the dataset.
- **Evaluation (or Target) Policy** ($\pi_e$): The new policy whose value we want to estimate.
- **Dataset** ($\mathcal{D}$): A collection of $N$ trajectories collected using $\pi_b$: $\displaystyle \mathcal{D} = \{\tau_i\}_{i=1}^N$.
- **Importance Ratio** ($\rho_t$): The ratio of probabilities for a single step: $\displaystyle \rho_t = \frac{\pi_e(a_t | s_t)}{\pi_b(a_t | s_t)}$.
- **Step-wise Ratio** ($\rho_{0:t}$): The product of importance ratios up to time $t$: $\displaystyle \rho_{0:t} = \prod_{k=0}^t \rho_k$.
- **Trajectory Ratio** ($\rho_{0:T}$): The product of importance ratios for a full trajectory: $\displaystyle \rho_{0:T} = \prod_{k=0}^T \rho_k$.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from dataclasses import dataclass

print("imports done!")

imports done!


In [84]:
class BehaviorPolicy(nn.Module):
    """pi_b(a|s): A simple network to clone a policy pi(a|s)."""
    def __init__(self, state_dim, num_actions):
        super(BehaviorPolicy, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 256), 
            nn.ReLU(),
            nn.Linear(256, 128), 
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )
    def forward(self, state):
        return self.net(state)  # return logits

    def get_probs(self, state, return_log=False):
        if return_log:
            return torch.log_softmax(self.forward(state), dim=-1)
        else:
            return torch.softmax(self.forward(state), dim=-1)
 

class TargetPolicy(nn.Module):
    def __init__(self, state_shape, n_actions, device="cpu", epsilon=0.1):
        """
        Initializes a DQN agent, which is the target policy.
        
        Args:
            state_shape (tuple): Shape of the input state.
            n_actions (int): Number of possible actions.
            epsilon (float): Exploration rate for epsilon-greedy policy.
        """
        super().__init__()
        self.epsilon = epsilon
        self.n_actions = n_actions
        self.state_shape = state_shape
        self.device = device
        
        # Define a simple feedforward neural network
        self.network = nn.Sequential(
            nn.Linear(state_shape[0], 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    
    def forward(self, states):
        """
        Forward pass through the network to get Q-values for given states.
        Args:
            states (array-like): Batch of states.
        Returns:
            torch.Tensor: Q-values for each action.
        """
        if isinstance(states, list):
            states = torch.tensor(np.array(states), device=self.device, dtype=torch.float32)
        if isinstance(states, np.ndarray):
            states = torch.tensor(np.array(states), device=self.device, dtype=torch.float32)
        elif isinstance(states, torch.Tensor):
            states = states.to(self.device).float()   # float32
        qvalues = self.network(states)
        return qvalues

    def get_action(self, states):
        """
        Returns the action as per epsilon-greedy policy for a batch of states.
        
        Args:
            states (array-like): Batch of states.
            
        Returns:
            np.array: Best actions.
        """
        qvalues = self.forward(states)
        random_numbers = np.random.rand()
        batch_size = qvalues.shape[0]
        if random_numbers < self.epsilon:
            random_actions = np.random.choice(self.n_actions, size=batch_size)
            return random_actions
        else:
            best_actions = qvalues.argmax(axis=-1)
            return best_actions
        
    def get_probs(self, states, return_log=False):
        if return_log:
            return torch.log_softmax(self.forward(states), dim=-1)
        else:
            return torch.softmax(self.forward(states), dim=-1)

print("Target policy and Behavior policy classes defined.")

Target policy and Behavior policy classes defined.


In [80]:
def dummy_dataset():
    # Dummy data creation
    # batch_size=5, number of timesteps=20, number of features=49
    # this means 5 trajectories of 20 timesteps each, with each timestep represented by a 49-dimensional feature vector
    states = torch.rand((5,20,49))
    next_states = torch.rand((5,20,49))
    # corresponding rewards for each timestep in each trajectory
    rewards = torch.rand((5,20,1))         
    # corresponding done flags for each timestep in each trajectory
    dones = torch.randint(0,2,(5,20,1))    
    # corresponding actions taken at each timestep in each trajectory
    actions = torch.randint(low=0, high=25, size=(5,20,1))
    return states, next_states, rewards, dones, actions


print("Dummy dataset function defined.")

Dummy dataset function defined.


### **IS: Importance Sampling**

**IS** is the most fundamental OPE method. It re-weights the returns from the behavior policy to look like they came from the evaluation policy.

It's based on a simple identity:


$$V(\pi_e) = \mathbb{E}_{\tau \sim \pi_e}[G(\tau)] = \int P(\tau | \pi_e) G(\tau) d\tau$$


We can multiply and divide by $P(\tau | \pi_b)$:


$$V(\pi_e) = \int \frac{P(\tau | \pi_e)}{P(\tau | \pi_b)} P(\tau | \pi_b) G(\tau) d\tau = \mathbb{E}_{\tau \sim \pi_b}\left[\frac{P(\tau | \pi_e)}{P(\tau | \pi_b)} G(\tau)\right]$$


The ratio of trajectory probabilities is:


$$\frac{P(\tau | \pi_e)}{P(\tau | \pi_b)} = \frac{\prod_{t=0}^T \pi_e(a_t | s_t) P(s_{t+1} | s_t, a_t)}{\prod_{t=0}^T \pi_b(a_t | s_t) P(s_{t+1} | s_t, a_t)} = \prod_{t=0}^T \frac{\pi_e(a_t | s_t)}{\pi_b(a_t | s_t)} = \rho_{0:T}$$

This gives the (Basic) Importance Sampling Estimator for $N$ trajectories:

$$\hat{V}_{IS} = \frac{1}{N} \sum_{i=1}^N \rho_{0:T}^{(i)} G^{(i)}$$

Log version of the estimator (to avoid numerical issues):

let $z^{(i)} = \log_e \rho_{0:T}^{(i)} + \log_e G^{(i)}$

$$\hat{V}_{IS} = \exp(\log_e \hat{V}_{IS}) = \exp\left(z_{\max} + \log_e\left(\sum_{i=1}^N \exp(z^{(i)} - z_{max})\right) - \log_e (N) \right)$$

where $z_{max} = \max{\{z^{(1)}, z^{(2)}, \ldots, z^{(N)}\}}$.

In [59]:
def fn():
    inputs = torch.arange(25, 34).reshape(3,3)
    print("Inputs:\n", inputs)
    idx1 = torch.tensor([0,1,2,1,2,0]).reshape(2,3)
    idx2 = torch.tensor([0,1,1,2,2,0]).reshape(3,2)
    print("Idx1:\n", idx1)
    print("Idx2:\n", idx2)
    gathered1 = inputs.gather(1, idx1)
    print("Gathered1:\n", gathered1)
    gathered2 = inputs.gather(0, idx2)
    print("Gathered2:\n", gathered2)
    gathered3 = inputs.gather(-2, idx2)
    print("Gathered3:\n", gathered3)
    gathered4 = inputs.gather(-1, idx1)
    print("Gathered4:\n", gathered4)


fn()

Inputs:
 tensor([[25, 26, 27],
        [28, 29, 30],
        [31, 32, 33]])
Idx1:
 tensor([[0, 1, 2],
        [1, 2, 0]])
Idx2:
 tensor([[0, 1],
        [1, 2],
        [2, 0]])
Gathered1:
 tensor([[25, 26, 27],
        [29, 30, 28]])
Gathered2:
 tensor([[25, 29],
        [28, 32],
        [31, 26]])
Gathered3:
 tensor([[25, 29],
        [28, 32],
        [31, 26]])
Gathered4:
 tensor([[25, 26, 27],
        [29, 30, 28]])


In [85]:
def is_estimate(states, actions, rewards, pi_b, pi_e, return_ir_sum=False):
    """
    Returns the importance sampling ratio between target policy and behavior policy.
    """
    pi_b_probs = pi_b.get_probs(states)
    pi_b_probs_actions = pi_b_probs.gather(2, actions)     # \pi_b(a_t | s_t)
    print("pi_b_probs.shape:\t\t", pi_b_probs.shape)
    print("pi_b_probs_actions.shape:\t", pi_b_probs_actions.shape)
    pi_e_probs = pi_e.get_probs(states)
    pi_e_probs_actions = pi_e_probs.gather(2, actions)     # \pi_e(a_t | s_t)
    print("pi_e_probs.shape:\t\t", pi_e_probs.shape)
    print("pi_e_probs_actions.shape:\t", pi_e_probs_actions.shape)
    importance_ratio = pi_e_probs / pi_b_probs             # \rho_{0:T}
    print("importance_ratio.shape:\t\t", importance_ratio.shape)
    print("rewards.shape:\t\t\t", rewards.shape)
    prod = torch.matmul(importance_ratio.transpose(1,2), rewards)          # \rho_{0:T}^{(i)} G^{(i)}
    print("prod.shape:\t\t\t", prod.shape)
    v_is = prod.mean().item()                              # \hat{V}_{IS}
    # print("importance_ratio.sum(dim=0).shape:\t", importance_ratio.sum(dim=0).shape)
    # print("importance_ratio.sum(dim=1).shape:\t", importance_ratio.sum(dim=1).shape)
    # print("importance_ratio.sum(dim=2).shape:\t", importance_ratio.sum(dim=2).shape)
    if return_ir_sum:
        # return both IS estimate and sum of importance ratios, which is needed for WIS
        return v_is, importance_ratio.sum(dim=1)
    else:
        return v_is



def fn():
    states, _, rewards, dones, actions = dummy_dataset()
    state_dim = states.shape[2]
    action_dim = 25
    pi_b = BehaviorPolicy(state_dim=state_dim, num_actions=action_dim)  # behavior policy (assumed it's trained)
    pi_e = TargetPolicy(state_shape=(state_dim,), n_actions=action_dim)     # target policy (assumed it's trained)
    v_is = is_estimate(states, actions, rewards, pi_b, pi_e)
    print("IS Estimate of V(pi_e):\t\t", v_is)


fn()

pi_b_probs.shape:		 torch.Size([5, 20, 25])
pi_b_probs_actions.shape:	 torch.Size([5, 20, 1])
pi_e_probs.shape:		 torch.Size([5, 20, 25])
pi_e_probs_actions.shape:	 torch.Size([5, 20, 1])
importance_ratio.shape:		 torch.Size([5, 20, 25])
rewards.shape:			 torch.Size([5, 20, 1])
prod.shape:			 torch.Size([5, 25, 1])
IS Estimate of V(pi_e):		 10.766754150390625


In [67]:
# def is_estimate_logversion():
#     """
#     Returns the importance sampling ratio between target policy and behavior policy using log probabilities.
#     """
#     states, rewards, dones, actions = dummy_dataset()
#     state_dim = states.shape[2]
#     action_dim = 25
#     pi_b = BehaviorPolicy(state_dim=state_dim, num_actions=action_dim)  # behavior policy (assumed it's trained)
#     pi_b_log_probs = pi_b.get_probs(states, return_log=True)
#     pi_b_log_probs_actions = pi_b_log_probs.gather(2, actions)
#     print(pi_b_log_probs.shape, pi_b_log_probs_actions.shape)
#     pi_e = DQNAgent(state_shape=(state_dim,), n_actions=action_dim)     # target policy (assumed it's trained)
#     pi_e_log_probs = pi_e.get_probs(states, return_log=True)
#     pi_e_log_probs_actions = pi_e_log_probs.gather(2, actions)
#     print(pi_e_log_probs.shape, pi_e_log_probs_actions.shape)
#     log_importance_ratio = pi_e_log_probs - pi_b_log_probs
#     print(log_importance_ratio.shape)
#     importance_ratio = torch.exp(log_importance_ratio)
#     print(importance_ratio.shape)
#     print(rewards.shape)
#     prod = torch.matmul(importance_ratio.transpose(1,2), rewards)
#     print(prod.shape)
#     v_is = prod.mean().item()
#     print("IS Estimate of V(pi_e) using log probs: ", v_is)


# is_estimate_logversion()

### **WIS: Weighted Importance Sampling**

WIS is a modification of IS that normalizes the weighted returns. This dramatically reduces variance, but introduces a small amount of bias.

Instead of dividing by $N$, WIS divides by the sum of the importance weights:

$$\hat{V}_{WIS} = \frac{\sum_{i=1}^N \rho_{0:T}^{(i)} G^{(i)}}{\sum_{i=1}^N \rho_{0:T}^{(i)}}$$

You can think of this as a weighted average, where the weight for trajectory $i$ is $\displaystyle w_i = \frac{\rho_{0:T}^{(i)}}{\sum_{j=1}^N \rho_{0:T}^{(j)}}$.

In [86]:
def wis_estimate(states, actions, rewards, pi_b, pi_e):
    """
    Returns the weighted importance sampling ratio between target policy and behavior policy.
    """
    v_is, ir_sum = is_estimate(states, actions, rewards, pi_b, pi_e, return_ir_sum=True)
    v_wis = v_is / ir_sum.mean().item()
    return v_wis


def fn():
    states, _, rewards, dones, actions = dummy_dataset()
    state_dim = states.shape[2]
    action_dim = 25
    pi_b = BehaviorPolicy(state_dim=state_dim, num_actions=action_dim)  # behavior policy (assumed it's trained)
    pi_e = TargetPolicy(state_shape=(state_dim,), n_actions=action_dim)     # target policy (assumed it's trained)
    v_wis = wis_estimate(states, actions, rewards, pi_b, pi_e)
    print("WIS Estimate of V(pi_e):\t", v_wis)

fn()

pi_b_probs.shape:		 torch.Size([5, 20, 25])
pi_b_probs_actions.shape:	 torch.Size([5, 20, 1])
pi_e_probs.shape:		 torch.Size([5, 20, 25])
pi_e_probs_actions.shape:	 torch.Size([5, 20, 1])
importance_ratio.shape:		 torch.Size([5, 20, 25])
rewards.shape:			 torch.Size([5, 20, 1])
prod.shape:			 torch.Size([5, 25, 1])
WIS Estimate of V(pi_e):	 0.5270333836347529


### **PDIS: Per-Decision Importance Sampling**

### **WPDIS: Weighted Per-Decision Importance Sampling**

### **FQE: Fitted Q-Evaluation**

FQE abandons importance sampling. Instead, it tries to learn the Q-function of the target policy, $Q^{\pi_e}(s, a)$, directly from the off-policy data.

FQE uses a batch RL algorithm, similar to Fitted Q-Iteration.

1. Initialize: Start with an arbitrary Q-function, $Q_0(s, a) = 0$.

2. Iterate ($k=1...K$):

- For every transition $(s_i, a_i, r_i, s'_i)$ in the dataset $\mathcal{D}$, create a target value $y_i$:


$$y_i = r_i + \gamma \mathbb{E}_{a' \sim \pi_e(a' | s'_i)}[Q_{k-1}(s'_i, a')]$$

- This expectation is just a sum: $y_i = r_i + \gamma \sum_{a'} \pi_e(a' | s'_i) Q_{k-1}(s'_i, a')$

- Train a function approximator (eg: a neural network) $Q_k$ to solve the regression problem: $Q_k \approx \text{argmin}_Q \sum_i (Q(s_i, a_i) - y_i)^2$.

3. Estimate Value: After $K$ iterations, the final $Q_K$ approximates $Q^{\pi_e}$. The policy value is the expected Q-value from the start states $s_0$:


$$\hat{V}_{FQE} = \mathbb{E}_{s_0 \sim \mathcal{D}, a_0 \sim \pi_e(a_0|s_0)}[Q_K(s_0, a_0)]$$

$$\hat{V}_{FQE} = \frac{1}{N} \sum_{i=1}^N \left( \sum_{a_0} \pi_e(a_0 | s_0^{(i)}) Q_K(s_0^{(i)}, a_0) \right)$$

In [ ]:
class QNet(nn.Module):
    def __init__(self, state_dim, num_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 256), 
            nn.ReLU(),
            nn.Linear(256, 128), 
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )
        
    def forward(self, x):
        return self.net(x)
    

print("QNet class defined.")

QNet class defined.


In [89]:
def fitted_q_evaluation(K, states, next_states, actions, rewards, dones, pi_e, gamma=0.99):
    """
    Fitted Q-evaluation using the given dataset and policy.

    Args:
        - K (int): Number of iterations.
        - states (torch.Tensor): Batch of states.
        - next_states (torch.Tensor): Batch of next states.
        - actions (torch.Tensor): Batch of actions taken.
        - rewards (torch.Tensor): Batch of rewards received.
        - dones (torch.Tensor): Batch of done flags.
        - pi_e (DQNAgent): Target policy.
        - gamma (float): Discount factor.

    Returns:
        - q_net (QNet): Trained Q-network.
    """
    qnet = QNet(state_dim=states.shape[2], num_actions=25)
    for k in range(K):
        # Update Q-values using the Bellman equation
        with torch.no_grad():
            expected_value = torch.sum(pi_e.get_probs(next_states) * qnet(next_states), dim=-1, keepdim=True)
            y = rewards + gamma * (1 - dones) * expected_value   # target Q-values
            qnet.update(states, actions, y)

    pass


def fn():
    states, next_states, rewards, dones, actions = dummy_dataset()
    state_dim = states.shape[2]
    action_dim = 25
    print("states.shape:\t\t", states.shape)
    print("next_states.shape:\t", next_states.shape)
    print("actions.shape:\t\t", actions.shape)
    print("rewards.shape:\t\t", rewards.shape)
    print("dones.shape:\t\t", dones.shape)
    pi_e = TargetPolicy(state_shape=(state_dim,), n_actions=action_dim)     # target policy (assumed it's trained)
    print("pi_e(next_states).shape:\t", pi_e.get_probs(next_states).shape)
    q_net = QNet(state_dim, action_dim)
    print("q_net(next_states).shape:\t", q_net(next_states).shape)
    expected_value = torch.sum(pi_e.get_probs(next_states) * q_net(next_states), dim=-1, keepdim=True)
    print("expected_value.shape:\t\t", expected_value.shape)

fn()

states.shape:		 torch.Size([5, 20, 49])
next_states.shape:	 torch.Size([5, 20, 49])
actions.shape:		 torch.Size([5, 20, 1])
rewards.shape:		 torch.Size([5, 20, 1])
dones.shape:		 torch.Size([5, 20, 1])
pi_e(next_states).shape:	 torch.Size([5, 20, 25])
q_net(next_states).shape:	 torch.Size([5, 20, 25])
expected_value.shape:		 torch.Size([5, 20, 1])


### **DR: Doubly Robust**

### **WDR: Weighted Doubly Robust**